In [1]:
from story_sage import StorySageConfig
from story_sage.utils import Raptor, Embedder
from openai import OpenAI
import yaml
import httpx
import os
import chromadb
from pprint import pprint


os.environ['TOKENIZERS_PARALLELISM'] = "false"

config_path = './config.yml'
raptor = Raptor(config_path=config_path)

ssconfig = StorySageConfig.from_file(config_path)

source_file = './books/harry_potter/01_hp_test.txt'

#client = OpenAI(api_key=ssconfig.openai_api_key, http_client=httpx.Client(verify=False))

#embedder = Embedder()

#chroma_client = chromadb.EphemeralClient()

In [2]:
results = raptor.process_texts(source_file, number_of_levels=3, target_dim=5, threshold=0.5)

Name: 01_hp_test.txt Book Number: 1
Book 1 has 4 chapters (0 indexed to include prologue).
Processing all chunks for book 01_hp_test.txt


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [3]:
if False:
    with open('hp_1_2.pkl', 'wb') as f:
        import pickle
        pickle.dump(results, f)

In [3]:
import copy

results_copy = copy.deepcopy(results)

In [10]:
levels = ['level_1_summaries','level_2_summaries','level_3_summaries','level_1_clusters','level_2_clusters','level_3_clusters']

for chapter, chapter_results in results_copy['01_hp_test.txt'].items():
    for level in levels:
        if level in chapter_results:
            _ = chapter_results[level].pop('embedding') if 'embedding' in chapter_results[level] else None
            print(f'{chapter} - {level}')
            pprint(chapter_results[level].to_dict(orient='records'))

chapter_0 - level_1_summaries
[{'book_number': 1,
  'chapter_number': 1,
  'chunk_ids': 'chunk_key|book_1|chapter_1|level_1|0',
  'cluster_ids': 'cluster_key|book_1|chapter_1|level_1|0',
  'clusters': 0,
  'level': 1,
  'summaries': '"Harry Potter and the Sorcerer’s Stone" is a book by J. K. '
               'Rowling, illustrated by Mary Grandpré, and published by Arthur '
               'A. Levine Books, an imprint of Scholastic Press. The book is '
               'dedicated to Jessica, who loves stories, Anne, who loved them '
               'too, and Di, who heard the story first.'}]
chapter_0 - level_1_clusters
[{'book_number': 1,
  'chapter_number': 1,
  'cluster_ids': ['cluster_key|book_1|chapter_1|level_1|0'],
  'clusters': array([0]),
  'texts': 'Harry Potter and the Sorcerer’s Stone by J. K. Rowling '
           'Illustrations by Mary Grandpré Arthur A. Levine Books An Imprint '
           'of Scholastic Press. For Jessica, who loves stories for Anne, who '
           'loved t

In [9]:
results_copy['02_the_chamber_of_secrets.txt']['chapter_1']['level_1_summaries'].to_dict().keys()

dict_keys(['summaries', 'level', 'clusters', 'cluster_ids', 'chunk_ids', 'book_number', 'chapter_number'])

In [13]:
import re
# get level number from format level_{number}_(clusters|summaries)
level_type_re = re.compile(r'level_(\d+)_(clusters|summaries)')

collected_chunks = []

class Chunk:
    def __init__(self, 
                 chunk_id: str, 
                 level: int, 
                 text: str, 
                 cluster_ids: str):
        
        self.chunk_id = chunk_id
        self.level = level
        self.text = text
        self.cluster_ids = cluster_ids
    
    def __str__(self):
        return f'Chunk {self.chunk_id} - Level {self.level} - Parents: {self.cluster_ids}'
    
    def __repr__(self):
        return self.__str__()

for file_name, chapters in results_copy.items():
    for chapter_name, data in chapters.items():
        for level, level_data in data.items():
            print(level_data.keys())
            level_number = int(level_type_re.match(level).group(1)) # Get 1, 2, 3, etc
            level_type = level_type_re.match(level).group(2) # Get clusters or summaries
            if (level_type not in ['summaries']) or \
                (level_type == 'summaries' and level_number == 1):
                continue
            text_key = 'texts' if level_type == 'clusters' else 'summaries'
            for i in range(len(level_data)):
                collected_chunks.append(
                    Chunk(
                        chunk_id=level_data['chunk_ids'][i],
                        level=int(level_number),
                        text=level_data[text_key][i],
                        cluster_ids=level_data['cluster_ids'][i]
                    )
                )
                    
            
                                        

Index(['texts', 'embedding', 'clusters', 'cluster_ids', 'book_number',
       'chapter_number'],
      dtype='object')
Index(['summaries', 'level', 'clusters', 'cluster_ids', 'chunk_ids',
       'book_number', 'chapter_number'],
      dtype='object')
Index(['texts', 'embedding', 'clusters', 'cluster_ids', 'book_number',
       'chapter_number'],
      dtype='object')
Index(['summaries', 'level', 'clusters', 'cluster_ids', 'chunk_ids',
       'book_number', 'chapter_number'],
      dtype='object')
Index(['texts', 'embedding', 'clusters', 'cluster_ids', 'book_number',
       'chapter_number'],
      dtype='object')
Index(['summaries', 'level', 'clusters', 'cluster_ids', 'chunk_ids',
       'book_number', 'chapter_number'],
      dtype='object')
Index(['texts', 'embedding', 'clusters', 'cluster_ids', 'book_number',
       'chapter_number'],
      dtype='object')
Index(['summaries', 'level', 'clusters', 'cluster_ids', 'chunk_ids',
       'book_number', 'chapter_number'],
      dtype='objec

In [14]:
collected_chunks

[Chunk chunk_key|book_1|chapter_2|level_2|0 - Level 2 - Parents: cluster_key|book_1|chapter_2|level_2|0,
 Chunk chunk_key|book_1|chapter_6|level_2|0 - Level 2 - Parents: cluster_key|book_1|chapter_6|level_2|0,
 Chunk chunk_key|book_1|chapter_7|level_2|0 - Level 2 - Parents: cluster_key|book_1|chapter_7|level_2|0,
 Chunk chunk_key|book_1|chapter_8|level_2|0 - Level 2 - Parents: cluster_key|book_1|chapter_8|level_2|0,
 Chunk chunk_key|book_1|chapter_10|level_2|0 - Level 2 - Parents: cluster_key|book_1|chapter_10|level_2|0,
 Chunk chunk_key|book_1|chapter_11|level_2|0 - Level 2 - Parents: cluster_key|book_1|chapter_11|level_2|0,
 Chunk chunk_key|book_1|chapter_13|level_2|0 - Level 2 - Parents: cluster_key|book_1|chapter_13|level_2|0,
 Chunk chunk_key|book_1|chapter_16|level_2|0 - Level 2 - Parents: cluster_key|book_1|chapter_16|level_2|0,
 Chunk chunk_key|book_1|chapter_17|level_2|0 - Level 2 - Parents: cluster_key|book_1|chapter_17|level_2|0,
 Chunk chunk_key|book_1|chapter_18|level_2|0 

In [13]:


def add_results_to_chroma(results, embedder, collection_name="raptor_collection"):
    # 1. Initialize client & embedding function
    client = chromadb.PersistentClient(path='./chroma_data')
    collection = client.create_collection(collection_name, embedding_function=embedder)

    class ChunkData:
        def __init__(self, book_number, chapter_number, chunk_id, level):
            self.text = text
            self.level = level

    # 2. Flatten results into a list of (text, level, parents, children)
    #    Below is a simple example schema; adapt to your actual data
    chunk_data = []
    index = 0
    for file_name, chapters in results.items():
        for chapter_name, data in chapters.items():
            level = 1
            get_next_chapter = False
            while not get_next_chapter:
                try:
                    cluster_data = data[f'level_{level}_clusters']
                    summary_data = data[f'level_{level}_summaries']
                    for i, cluster in cluster_data.iterrows():
                        chunk_data.append((cluster['text'], index, {
                            'file_name': file_name,
                            'chapter_name': chapter_name,
                            'level': level,
                            'cluster_id': i
                        }, []))
                        index += 1
                    level += 1
                except KeyError:
                    get_next_chapter = True
                
            
    # 3. Add flattened data to collection
    documents = [cd[0] for cd in chunk_data]
    ids = [cd[1] for cd in chunk_data]
    metadatas = [cd[2] for cd in chunk_data]

    collection.add(
        documents=documents,
        ids=ids,
        metadatas=metadatas
    )
    return collection

,texts,embedding,clusters,book_number,chapter_number
0,"At the Weasleys' home, life is chaotic and vib...","[0.034855302, 0.040605072, 0.022834748, 0.0369...",[0],2,5
1,"Ron receives a letter from Hermione via Errol,...","[0.043866813, 0.016107008, 0.013852521, 0.0356...",[0],2,5
2,Harry hides in a large black cabinet as Draco ...,"[-0.023570418, 0.010350927, 0.0035565388, 0.06...",[0],2,5
3,Harry and Hagrid meet after Harry gets lost. H...,"[0.018442497, -0.010843633, 0.016544182, 0.012...",[0],2,5
4,"In a visit to Diagon Alley, the Weasley family...","[0.03071635, 0.026746025, 0.00867683, 0.063738...",[0],2,5


In [10]:
collection.add(ids=raptor_dict['ids'], embeddings=raptor_dict["embeddings"], documents=raptor_dict["texts"], metadatas=raptor_dict["metadatas"])

In [11]:
def generate_results(
    query : str,
    context_text : str
) -> str:

    developer_prompt = f"""
    You are a helpful GPT named Story Sage. Although you are not the author, you should use the voice of an author talking about their own work.
    
    # Your Goal
    You need to help a reader keep track of people, places, and plot points in books.
    The reader is currently reading a book series and has provided you with a question about the context of the book.
    You will be provided with text delimited by triple quotes that contain context from the book.

    # Your Task
    Based on the context and question provided:

    1 - Think about the what information you would need to answer their question.
    2 - Review the context provided to find supporting information for your answer.
    3 - Be skeptical--don't just find supporting information, but also consider that the user may be misleading you.

    If the answer isn't clearly stated in the context, don't guess. DO NOT INCLUDE INFORMATION THAT DOESN'T APPEAR IN THE CONTEXT.
    If the question mentions specific people, places, or things, but they do not appear in the context, don't assume anything about them.

    # Your Output
    1 - Provide a sentence or two that answers the reader's question based on the context provided.
    2 - Provide bullet points with excerpts from the context that support your answer.

    The reader is in Book 1 and you MAY NOT GIVE SPOILERS, so only use information from the context provided. Do not include information from the broader universe.

    """

    user_prompt = f"""
    # Question
    {query}

    # Context
    {context_text}
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=[
            {"role": "developer", "content": developer_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=500,
        n=1,
        stop=None,
        temperature=0.7
    )
    answer = response.choices[0].message.content.strip()
    return answer

In [12]:
with open('tests/test_config.yml', 'r') as f:
    test_config = yaml.safe_load(f)

In [13]:
def get_answer(question, chapter):
    search_results = collection.query(query_texts=question, n_results=20, where={"chapter": { '$lt': chapter + 1 }})
    context_text = "------\n\n".join(set(search_results['documents'][0]))
    raptor_answer = generate_results(question, context_text)
    print(f"Question: {question}")
    print()
    print(f"Answer: {raptor_answer}")
    print()
    print('💡' * 20)
    print()


## Results for the Whole Book

In [14]:
for question in test_config[0]['question_list']:
    get_answer(question, 18)

Question: Why does Dumbledore decide to have Harry grow up with the Dursleys rather than with one of the wizard families? How does Harry’s experience with his relatives build his character?

Answer: Dumbledore decides to have Harry grow up with the Dursleys to protect him from the wizarding world until he is ready, believing it is best for him to remain away from the fame and expectations associated with his identity as "the boy who lived." This experience with the Dursleys, who treat him poorly, builds his character by instilling resilience and a sense of empathy towards others who are marginalized.

- "Dumbledore insists it's best for him to grow up away from that world until he's ready."
- "Hagrid expresses anger towards the Dursleys for hiding Harry's magical heritage from him."
- Harry faces a harsh summer with his cousin Dudley and his gang tormenting him daily, which highlights his resilience.
- "The Dursleys go to extreme lengths to prevent Harry from receiving his letters," sh

## Results at Chapter 2

In [15]:
for question in test_config[0]['question_list']:
    get_answer(question, 2)

Question: Why does Dumbledore decide to have Harry grow up with the Dursleys rather than with one of the wizard families? How does Harry’s experience with his relatives build his character?

Answer: Dumbledore decides to leave Harry with the Dursleys because he believes it's the best place for him to grow up away from the wizarding world until he is ready to understand his legacy. Harry’s experience with the Dursleys, who treat him poorly, helps to build his character by fostering resilience and empathy, despite the neglect and bullying he endures.

- Dumbledore states, "It’s the best place for him... His aunt and uncle will be able to explain everything to him when he’s older."
- Dumbledore emphasizes that growing up away from fame and expectations will be beneficial for Harry: "Famous before he can walk and talk! Famous for something he won’t even remember! Can’t you see how much better off he’ll be, growing up away from all that until he’s ready to take it?"
- Harry's life with the 